In [ ]:
###登录脸书###
import traceback
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#关闭chrome弹窗通知，页面化时使用
prefs = {
    'profile.default_content_setting_values' :
        {
        'notifications' : 2
         }
}
chrome_options = Options()
#无可视化界面禁用
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--no-sandbox') #解决DevToolsActivePort文件不存在的报错
chrome_options.add_argument("disable-infobars") # 关闭'chrome正受到自动测试软件的控制'提示
chrome_options.add_argument('window-size=1920x3000')  #指定浏览器分辨率
chrome_options.add_argument('--disable-gpu')  #谷歌文档提到需要加上这个属性来规避bug
chrome_options.add_argument('--hide-scrollbars')  #隐藏滚动条, 应对一些特殊页面
chrome_options.add_argument('blink-settings=imagesEnabled=false')  #不加载图片, 提升速度

chrome_options.add_argument('--headless')  #浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
chrome_options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" #手动指定本机电脑使用的浏览器位置

# 创建driver,进行后面的请求页面等操作，executable_path指定本机中chromedriver.exe的位置
driver = webdriver.Chrome(options=chrome_options, executable_path=r"C:\Users\UNclePeach\AnacondaProjects\Social Media CRM Analysis\chromedriver.exe") 
# 使用driver去请求页面
driver.get("https://www.facebook.com/")

# driver去填写用户名和密码进行模拟登陆
driver.find_element_by_id('email').send_keys("taojing@csu.fullerton.edu")
driver.find_element_by_id('pass').send_keys("Lettopia@20190806")
# driver自动点击登陆按钮
driver.find_element_by_id('loginbutton').click()

In [ ]:
####df读取csv数据库发送marketplace问询信息后更新csv
import pandas as pd

driver.set_page_load_timeout(30)

items = pd.read_csv("item.csv")
for i in range(0,len(items)):
    if items.iloc[i]["status"] == "no contact":
        driver.get(items.iloc[i]["url"])
        try:
            driver.find_element_by_xpath('//span[contains(text(), "This Item Isn\'t Available Anymore")]')
            items.iloc[j,2] = "item unavailable"
            print(i, "This Item Isn't Available Anymore!")
        except:
            try:
                message = driver.find_element_by_xpath("//input[@placeholder = 'Send a private message...']")
                message.send_keys(u'\ue009' + "a")
                ###针对手机显示屏维修业务发出询问
                message.send_keys("Hello, I'm interested in your item. Do you repair iPhoneX screen?")
                send = driver.find_element_by_xpath("//div[contains(text(), 'Send')]")
                send = send.find_element_by_xpath("..//..")
                send = send.find_element_by_xpath("..")
                #send.click()
                time.sleep(10)
                print(i,str(items.iloc[i]["seller_id"]) + "@send")
                #print(i,items.iloc[i]["url"])
                items.iloc[i,2] = "sent"
                #print(i,items.iloc[i]["seller_id"],items.iloc[i]["status"])
                for j in range(i+1,len(items)):
                    if items.iloc[j]["seller_id"] == items.iloc[i]["seller_id"]:
                #print(j,items.iloc[j]["seller_id"],items.iloc[j]["status"])
                       items.iloc[j,2] = "already sent"
                #print(j,items.iloc[j,0],items.iloc[j,2])
                    else:
                        continue
                #拉长休眠时间，循环发送消息，一天以30条为max      
                #time.sleep(2860)
                #driver.refresh()
                #time.sleep(10)
            except:
                f=open("log.txt",'a')  
                traceback.print_exc(file=f)  
                f.flush()  
                f.close()
                print(i, str(items.iloc[i]["seller_id"]) + "@error")
            continue

In [ ]:
items.to_csv('item_n.csv',index=False)

In [ ]:
####检查信息发送状态准确性
items = pd.read_csv("item_n.csv")
for i in range(0,len(items)):
    if items.iloc[i]["status"] == "sent":
        driver.get(items.iloc[i]["url"])
        time.sleep(15)
        try:
            sent = driver.find_element_by_xpath('//button[contains(text(), "Message Again")]')        
            print(i,str(items.iloc[i]["seller_id"]) + "@correct")
        except:
            f=open("log.txt",'a')  
            traceback.print_exc(file=f)  
            f.flush()  
            f.close()
            if items.iloc[i]["status"] != "item unavailable":
                print(i, str(items.iloc[i]["url"]) + "@please check again")
        continue

In [ ]:
driver.quit()

In [ ]:
import pandas as pd
items = pd.read_csv(r"C:\Users\UNclePeach\Downloads\item.csv", encoding = "ISO-8859-1")